In [9]:
pip install ultralytics

  Using cached skimage-0.0.tar.gz (757 bytes)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [3 lines of output]
  
  *** Please install the `scikit-image` package (instead of `skimage`) ***
  
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


# 1st Version | Working Version 

In [1]:
import math


class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
#                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [37]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import *


model=YOLO('yolov8s.pt')


def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap=cv2.VideoCapture(r"C:\Users\abdul\Desktop\peoplecount2.mp4")


my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")
#print(class_list)
count=0
tracker=Tracker()   

          #dl           #dr          #ul        #ur  
area1 = [(180, 284), (294, 266,), (313, 291), (197, 311)]

area2= [(483, 264), (390, 285), (361, 261), (476, 244)]

area1_c = set()
area2_c = set()

while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue


    frame=cv2.resize(frame,(1020,500))

    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.boxes
    px=pd.DataFrame(a).astype("float")
#    print(px)
    list=[]
    for index,row in px.iterrows():
#        print(row)
 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        list.append([x1,y1,x2,y2])
            
    bbox_idx=tracker.update(list)
    for bbox in bbox_idx:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
        results = cv2.pointPolygonTest(np.array(area1, np.int32),(x4,y4),False)
        if results >=0:
            cv2.rectangle(frame,(x3,y3),(x4,y4),(0,255,0),2)
            cv2.circle(frame,(x4,y4),5,(255,0,255),-1)
            cv2.putText(frame,str(int(id)),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,0,0),1)
            area1_c.add(id)
        results1 = cv2.pointPolygonTest(np.array(area2, np.int32),(x4,y4),False)
        if results1 >=0:
            cv2.rectangle(frame,(x3,y3),(x4,y4),(0,255,0),2)
            cv2.circle(frame,(x4,y4),5,(255,0,255),-1)
            cv2.putText(frame,str(int(id)),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,0,0),1)
            area2_c.add(id)
            
    up = ('UP' ,  len(area1_c))
    down = ('DOWN' , len(area2_c))
    
    cv2.putText(frame,str(up),(50,80),cv2.FONT_HERSHEY_PLAIN,3,(255,255,255),1)
    cv2.putText(frame,str(down),(50,160),cv2.FONT_HERSHEY_PLAIN,3,(255,255,255),1)
    
    
    cv2.polylines(frame, [np.array(area1, np.int32)], True, (0, 0, 225), 2)
    cv2.polylines(frame, [np.array(area2, np.int32)], True, (0, 0, 225), 2)  
    cv2.imshow("RGB", frame)
    
    # Define key function to exit program by pressing 'q'
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
        
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()


0: 320x640 24 persons, 183.1ms
Speed: 1.0ms preprocess, 183.1ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)



[993, 1]
[406, 237]


0: 320x640 25 persons, 208.5ms
Speed: 1.0ms preprocess, 208.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)



[712, 13]


0: 320x640 22 persons, 197.5ms
Speed: 0.5ms preprocess, 197.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)



...

0: 320x640 34 persons, 1 bicycle, 184.9ms
Speed: 1.0ms preprocess, 184.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)



[696, 277]


0: 320x640 28 persons, 2 bicycles, 191.0ms
Speed: 1.0ms preprocess, 191.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 32 persons, 1 bicycle, 193.5ms
Speed: 0.0ms preprocess, 193.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)



[935, 0]
[935, 1]


0: 320x640 27 persons, 1 bicycle, 187.2ms
Speed: 0.0ms preprocess, 187.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 28 persons, 1 bicycle, 197.5ms
Speed: 0.0ms preprocess, 197.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 31 persons, 1 bicycle, 180.6ms
Speed: 1.0ms preprocess, 180.6ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 27 persons, 1 bicycle, 191.7ms
Speed: 1.0ms preprocess, 191.7ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 27 persons, 1 bicycle, 200.7ms
Speed: 2.0ms preprocess, 200.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 25 persons, 1 bicycle, 200.7ms
Speed: 1.0ms preprocess, 200.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 26 persons, 1 bicycle, 228.6ms
Speed: 1.3ms preprocess, 228.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 320x640 28

# 2nd Working Project 

In [3]:
from random import randint
import time

class MyPerson:
    tracks = []
    def __init__(self, i, xi, yi, max_age):
        self.i = i
        self.x = xi
        self.y = yi
        self.tracks = []
        self.R = randint(0,255)
        self.G = randint(0,255)
        self.B = randint(0,255)
        self.done = False
        self.state = '0'
        self.age = 0
        self.max_age = max_age
        self.dir = None
    def getRGB(self):
        return (self.R,self.G,self.B)
    def getTracks(self):
        return self.tracks
    def getId(self):
        return self.i
    def getState(self):
        return self.state
    def getDir(self):
        return self.dir
    def getX(self):
        return self.x
    def getY(self):
        return self.y
    def updateCoords(self, xn, yn):
        self.age = 0
        self.tracks.append([self.x,self.y])
        self.x = xn
        self.y = yn
    def setDone(self):
        self.done = True
    def timedOut(self):
        return self.done
    def going_UP(self,mid_start,mid_end):
        if len(self.tracks) >= 2:
            if self.state == '0':
                if self.tracks[-1][1] < mid_end and self.tracks[-2][1] >= mid_end: #cruzo la linea
                    state = '1'
                    self.dir = 'up'
                    return True
            else:
                return False
        else:
            return False
    def going_DOWN(self,mid_start,mid_end):
        if len(self.tracks) >= 2:
            if self.state == '0':
                if self.tracks[-1][1] > mid_start and self.tracks[-2][1] <= mid_start: #cruzo la linea
                    state = '1'
                    self.dir = 'down'
                    return True
            else:
                return False
        else:
            return False
    def age_one(self):
        self.age += 1
        if self.age > self.max_age:
            self.done = True
        return True
class MultiPerson:
    def __init__(self, persons, xi, yi):
        self.persons = persons
        self.x = xi
        self.y = yi
        self.tracks = []
        self.R = randint(0,255)
        self.G = randint(0,255)
        self.B = randint(0,255)
        self.done = False
        

In [4]:
##People counter
import numpy as np
import cv2
import time



cnt_up   = 0
cnt_down = 0
count_up = 0
count_down = 0
state =0

#Taking the video input
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(r"C:\Users\abdul\Downloads\stock-footage-skolkovo-russia-april-time-lapse-high-angle-of-various-people-passing-through.webm")
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output1.mkv',fourcc, 20.0, (640,480))



##cap.set(3,160) #Width
##cap.set(4,120) #Height

#Print the capture properties to console
for i in range(19):
    print (i, cap.get(i))

w = cap.get(3)
h = cap.get(4)
frameArea = h*w
areaTH = frameArea/300
print ('Area Threshold', areaTH)

#Lines coordinate for counting
line_up = int(2*(h/5))
line_down   = int(4*(h/5))

up_limit =   int(.5*(h/5))
down_limit = int(4.5*(h/5))

print ("Red line y:",str(line_down))
print ("Blue line y:", str(line_up))
line_down_color = (255,0,0)
line_up_color = (0,0,255)

pt1 =  [1, line_down];
pt2 =  [w, line_down];
pts_L1 = np.array([pt1,pt2], np.int32)
pts_L1 = pts_L1.reshape((-1,1,2))
pt3 =  [0, line_up];
pt4 =  [w, line_up];

pts_L2 = np.array([pt3,pt4], np.int32)
pts_L2 = pts_L2.reshape((-1,1,2))

pt5 =  [0, up_limit];
pt6 =  [w, up_limit];

pts_L3 = np.array([pt5,pt6], np.int32)
pts_L3 = pts_L3.reshape((-1,1,2))
pt7 =  [0, down_limit];
pt8 =  [w, down_limit];
pts_L4 = np.array([pt7,pt8], np.int32)
pts_L4 = pts_L4.reshape((-1,1,2))

#Background Substractor
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows = True)

#Structuring elements for morphographic filters
kernelOp = np.ones((3,3),np.uint8)
kernelOp2 = np.ones((5,5),np.uint8)
kernelCl = np.ones((11,11),np.uint8)

#Variables
font = cv2.FONT_HERSHEY_SIMPLEX
persons = []
rect_co = []
max_p_age = 1
pid = 1
val = []

while(cap.isOpened()):
##for image in camera.capture_continuous(rawCapture, format="bgr", use_video_port=True):
    
    ret, frame = cap.read()
##    frame = image.array

    # for i in persons:
    #     print i.age_one() #age every person one frame

    
    
    #Apply background subtraction
    fgmask = fgbg.apply(frame)
    fgmask2 = fgbg.apply(frame)

    #Binarization to eliminate shadows
    try:
        ret,imBin= cv2.threshold(fgmask,200,255,cv2.THRESH_BINARY)
        ret,imBin2 = cv2.threshold(fgmask2,200,255,cv2.THRESH_BINARY)
        #Opening (erode->dilate) to remove noise.
        mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernelOp)
        mask2 = cv2.morphologyEx(imBin2, cv2.MORPH_OPEN, kernelOp)
        #Closing (dilate -> erode) to join white regions.
        mask =  cv2.morphologyEx(mask , cv2.MORPH_CLOSE, kernelCl)
        mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernelCl)
    except:
        print('EOF')
        print ('UP:',cnt_up+count_up)
        print ('DOWN:',cnt_down+count_down)
        break
    #################
    #   CONTOURS   #
    #################
    
    # RETR_EXTERNAL returns only extreme outer flags. All child contours are left behind.
    contours0, hierarchy = cv2.findContours(mask2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours0:
        rect = cv2.boundingRect(cnt)
        # print rect_co
        # if rect[2] > 100: 
        #     if rect[1]!=0:
        #         rect_co.append(rect[1])
        #     if len(rect_co)>=2:
        #         if (rect_co[-1]-rect_co[-2]) > 0:
        #             count_down = rect[2]/60
        #             count_up = 0
        #             print 'down' ,count_down
        #         elif (rect_co[-1]-rect_co[-2]) < 0:
        #             count_up =  rect[2]/60
        #             count_down = 0
        #             print 'up',count_up
                
        #     continue
        area = cv2.contourArea(cnt)
        if area > areaTH:
            #################
            #   TRACKING    #
            #################
            
            #Missing conditions for multipersons, outputs and screen entries
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            x,y,w,h = cv2.boundingRect(cnt)
            # print 'working'
            # print w

            new = True
            if cy in range(up_limit,down_limit):
                for i in persons:
                    if abs(cx-i.getX()) <= w and abs(cy-i.getY()) <= h:
                        # the object is close to one that has already been detected before
                        # print 'update'
                        new = False
                        i.updateCoords(cx,cy)   #update coordinates in the object and resets age
                        if i.going_UP(line_down,line_up) == True:
                            if w > 100:
                                count_up = w/60
                                #cnt_up += count_up
                                print ()
                            else:    
                                cnt_up += 1;
                            print ("ID:",i.getId(),'crossed going up at',time.strftime("%c"))
                        elif i.going_DOWN(line_down,line_up) == True:
                            if w > 100:
                                count_down = w/60
                                #cnt_down += count_down
                            else:
                                cnt_down += 1;
                            print ("ID:",i.getId(),'crossed going down at',time.strftime("%c"))
                        break
                    if i.getState() == '1':
                        if i.getDir() == 'down' and i.getY() > down_limit:
                            i.setDone()
                        elif i.getDir() == 'up' and i.getY() < up_limit:
                            i.setDone()
                    if i.timedOut():
                        #get out of the people list
                        index = persons.index(i)
                        persons.pop(index)
                        del i     #free the memory of i
                if new == True:
                    p = MyPerson(pid,cx,cy, max_p_age)
                    persons.append(p)
                    pid += 1    
            # new = True
            # print cy
            # if cy in range(up_limit,down_limit):
            #     for i in persons:
            #         if abs(cx-i.getX()) <= w and abs(cy-i.getY()) <= h:
            #             # the object is close to one that has already been detected before
            #             new = False
            #             i.updateCoords(cx,cy)
            #             val = i.getTracks()   #update coordinates in the object and resets age
            #             print val
                    
            #     # print new  
            #     if new == True:
            #         p = Person.MyPerson(pid,cx,cy, max_p_age)
            #         persons.append(p)
            #         pid += 1 
            #     # print 'person length',len(persons)     
            #     if len(val)>=2:
            #         if (val[-1][1]-val[-2][1]) > 0:
            #             cnt_down += 1;
            #             state='1'
            #             getdir = 'down'
            #             # print "ID:",i.getId(),'crossed going up at',time.strftime("%c")
            #         elif (val[-1][1]-val[-2][1]) < 0:
            #             cnt_up += 1;
            #             state = '1'
            #             getdir = 'up'
            #             # print "ID:",i.getId(),'crossed going down at',time.strftime("%c")
            #         val = []    
            #         if state == '1':
            #                 if getdir == 'down':
            #                     done=True
            #                 elif getdir == 'up':
            #                     done = True
            #         if done:
            #              #get out of the people list
            #             j=persons[0]
            #             # print j
            #             index = persons.index(j)
            #             persons.pop(index)
            #             # print "delete"
            #             del j     #free the memory of i
            #################
            #   DRAWINGS     #
            #################
            cv2.circle(frame,(cx,cy), 5, (0,0,255), -1)
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)            
            #cv2.drawContours(frame, cnt, -1, (0,255,0), 3)
            
    #END for cnt in contours0
            
    #########################
    # DRAWING TRAJECTORIES  #
    #########################
    for i in persons:
##        if len(i.getTracks()) >= 2:
##            pts = np.array(i.getTracks(), np.int32)
##            pts = pts.reshape((-1,1,2))
##            frame = cv2.polylines(frame,[pts],False,i.getRGB())
##        if i.getId() == 9:
##            print str(i.getX()), ',', str(i.getY())
        cv2.putText(frame, str(i.getId()),(i.getX(),i.getY()),font,0.3,i.getRGB(),1,cv2.LINE_AA)
        
    #################
    # DISPLAY ON FRAME    #
    #################
    str_up = 'UP: '+ str(cnt_up+count_up)
    str_down = 'DOWN: '+ str(cnt_down+count_down)
    
    frame = cv2.polylines(frame,[pts_L1],False,line_down_color,thickness=2)
    frame = cv2.polylines(frame,[pts_L2],False,line_up_color,thickness=2)
    
    
    
    frame = cv2.polylines(frame,[pts_L3],True,(255,255,255),thickness=1)
    frame = cv2.polylines(frame,[pts_L4],False,(255,255,255),thickness=1)
    cv2.putText(frame, str_up ,(10,40),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str_up ,(10,40),font,0.5,(0,0,255),1,cv2.LINE_AA)
    cv2.putText(frame, str_down ,(10,90),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str_down ,(10,90),font,0.5,(255,0,0),1,cv2.LINE_AA)
   

    
    out.write(frame)
    cv2.imshow('Frame',frame)
    #cv2.imshow('Mask',mask)    
    
    #Press ESC to exit
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
#END while(cap.isOpened())
    
#################
#   CLOSING    #
#################
cap.release()
cv2.destroyAllWindows()

0 0.0
1 0.0
2 0.001
3 596.0
4 336.0
5 29.97
6 808996950.0
7 240.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 1.0
17 0.0
18 0.0
Area Threshold 667.52
Red line y: 268
Blue line y: 134
ID: 2 crossed going down at Wed May 10 23:45:18 2023
ID: 1 crossed going up at Wed May 10 23:45:19 2023
ID: 1 crossed going up at Wed May 10 23:45:19 2023
ID: 7 crossed going up at Wed May 10 23:45:19 2023
ID: 1 crossed going down at Wed May 10 23:45:19 2023
ID: 4 crossed going down at Wed May 10 23:45:19 2023
ID: 3 crossed going down at Wed May 10 23:45:19 2023
ID: 2 crossed going down at Wed May 10 23:45:20 2023
ID: 1 crossed going up at Wed May 10 23:45:21 2023
ID: 3 crossed going up at Wed May 10 23:45:21 2023
ID: 6 crossed going down at Wed May 10 23:45:22 2023
ID: 7 crossed going up at Wed May 10 23:45:22 2023
ID: 1 crossed going down at Wed May 10 23:45:22 2023
ID: 6 crossed going down at Wed May 10 23:45:23 2023
ID: 3 crossed going up at Wed May 10 23:45:23 2023
ID: 1 crossed going up 

# 3rd Working Project 

In [5]:
import math


class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
#                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return 

In [6]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO



model=YOLO('yolov8s.pt')


area1=[(616, 235),(730, 250),(727, 280),(606, 265)]

area2=[(567, 274),(739, 302),(736, 315),(556, 293)]

def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap=cv2.VideoCapture(r'C:\Users\abdul\Downloads\Commercial CCTV at a Bar in Chapel Allerton, Leeds, LS7.mp4')


my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n") 
#print(class_list)

count=0

tracker = Tracker()
people = {}

while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 2 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
#    frame=cv2.flip(frame,1)
    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.boxes
    px=pd.DataFrame(a).astype("float")
#    print(px)
    list1=[]
             
    for index,row in px.iterrows():
#        print(row)
 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'person' in c:
            list1.append([x1, y1, x2, y2])
    bbox_id = tracker.update(list1)
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        results = cv2.pointPolygonTest(np.array(area2,np.int32),((x4,y4)),False) 
                result1 = cv2.pointPolygonTest(np.array(area2,np.int32),((x4,y4)),False) 
              
                    cv2.rectangle(frame,(x3,y3),(x4,y4),(0,255,0),2)
                    cv2.circle(frame,(x4,y4), 5, (255,0,255), -1)
                    cv2.putText(frame,str(id),(x3,y3),cv2.FONT_HERSHEY_COMPLEX,0.5,(255,255,255),1)

            
        
    cv2.polylines(frame,[np.array(area1,np.int32)],True,(255,0,0),2)
    cv2.putText(frame,str('1'),(504,471),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)

    cv2.polylines(frame,[np.array(area2,np.int32)],True,(255,0,0),2)
    cv2.putText(frame,str('2'),(466,485),cv2.FONT_HERSHEY_COMPLEX,(0.5),(0,0,0),1)
    print(people)

    cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF==27:
        break

cap.release()
cv2.destroyAllWindows()

IndentationError: unexpected indent (4073684311.py, line 68)

# 4th project 

In [1]:
## WORKING SPECS:
# OpenCV_3.1.0
# Python_2.7.12

import numpy as np
import cv2
import colorsys
import collections


# ====================================================
# ================== DEFINE CLASS ====================
# ====================================================

class Position(object):
    def __init__(self, _x, _y, _w, _h):
        self.x = _x
        self.y = _y
        self.w = _w
        self.h = _h

    def x(self):
        return self.x

    def y(self):
        return self.y

    def w(self):
        return self.w

    def h(self):
        return self.h

class People(object):
    def __init__(self, _x, _y, _w, _h, _roi, _hue):
        # Position
        self.x = _x
        self.y = _y
        self.w = _w
        self.h = _h
        self.roi = _roi

        # Display of the contour while tracking
        self.hue = _hue
        self.color = hsv2rgb(self.hue%1, 1, 1)

        # Motion Descriptors
        self.center = [_x + _w/2, _y + _h/2]
        self.isIn = checkPosition(boundaryPt1, boundaryPt2, self.center, inCriterion)
        self.isInChangeFrameCount = toleranceCountIOStatus
        self.speed = [0,0]
        self.missingCount = 0

        # ROI - Region of Interest
        self.maxRoi = _roi
        self.roi = _roi

    def x(self):
        return self.x

    def y(self):
        return self.y

    def w(self):
        return self.w

    def h(self):
        return self.h

    def roi(self):
        return self.roi

    def color(self):
        return self.color

    def center(self):
        return self.center

    def maxRoi(self):
        return self.maxRoi

    def isIn(self):
        return self.isIn

    def speed(self):
        return self.speed

    def missingCount(self):
        return self.missingCount

    def isInChangeFrameCount(self):
        return self.isInChangeFrameCount

    def set(self, name, value):
        if name == "x":
            self.x = value
        elif name == "y":
            self.y = value
        elif name == "w":
            self.w = value
        elif name == "h":
            self.h = value
        elif name == "center":
            self.center = value
        elif name == "roi":
            self.roi = value
            # Automatically update maxRoi as roi is updated
            if self.roi.shape[0]*self.roi.shape[1] > self.maxRoi.shape[0]*self.maxRoi.shape[1]:
                self.maxRoi = self.roi
        elif name == "speed":
            self.speed = value
        elif name == "missingCount":
            self.missingCount = value
        elif name == "isIn":
            self.isIn = value
        elif name == "isInChangeFrameCount":
            self.isInChangeFrameCount = value
        else:
            return


# ====================================================
# ===================== FUNCTION =====================
# ====================================================

def averageSize():
    sum = 0
    for i in humanSizeSample:
        sum +=i
    return sum/sampleSize


# Only care about top and bottom
def checkTouchVSide(x, y, w, h, maxW, maxH, tolerance):
    if x <= 0:
        return True
    elif y - tolerance <= 0:
        return True
    elif x + w >= maxW:
        return True
    elif y + h + tolerance >= maxH:
        return True
    else:
        return False


def getExteriorRect(pts):
    xArray = []
    yArray = []
    for pt in pts:
        xArray.append(pt[0])
        yArray.append(pt[1])
    xArray = sorted(xArray)
    yArray = sorted(yArray)
    return (xArray[0], yArray[0], xArray[3] - xArray[0], yArray[3] - yArray[0])


def hsv2rgb(h, s, v):
    return tuple(i * 255 for i in colorsys.hsv_to_rgb(h, s, v))


def checkPosition(boundaryPt1, boundaryPt2, currPos, inCriterion):
    m = (boundaryPt2[1] - boundaryPt1[1])/(boundaryPt2[0] - boundaryPt1[0])
    c = boundaryPt2[1] - m*boundaryPt2[0]
    if inCriterion == "<":
        if currPos[0] * m + c < currPos[1]:
            return True
        else:
            return False
    elif inCriterion == ">":
        if currPos[0] * m + c > currPos[1]:
            return True
        else:
            return False
    else:
        return False

def nothing(x):
    pass

# ====================================================
# ================== VIDEO SOURCE ====================
# ====================================================

srcTest = 'peopleCounter.avi'
srcWebcam = 0
srcMain = '' # live source here
cap = cv2.VideoCapture(srcTest)  # Open video file


# ====================================================
# ================== PRE-CONFIG ======================
# ====================================================

minArea = 500  # default min area to be considered person
maxArea = 4000  # default max area to be considered person
noFrameToCollectSample = 100
toleranceRange = 50  # use for error calculation
toleranceCount = 10  # maximum number of frame an object need to present in order to be accepted
toleranceCountIOStatus = 3  # minimum number of frame between In/Out Status change -> prevent playing with the system
startHue = 0  # In HSV this is RED
hueIncrementValue = 0.1  # increment color every time to differentiate between different people


# ====================================================
# ====================== SETUP =======================
# ====================================================

# fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold = 16, detectShadows=True)
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)
sampleSize = 100
humanSizeSample = collections.deque(maxlen=sampleSize)

midHeight = int(cap.get(4) / 2)
maxWidth = cap.get(3)
maxHeight = cap.get(4)

inCriterion = "<"
boundaryPt1 = [0, midHeight-100]
boundaryPt2 = [maxWidth, midHeight]


# ====================================================
# ====================== MAIN ========================
# ====================================================

# Passage Control
allowPassage = True
peopleViolationIn = 0
peopleViolationOut = 0
switch = '0 : PASS \n1 : STOP'

# Controller
cv2.namedWindow('config')
cv2.createTrackbar(switch, 'config', 0, 1, nothing)

# Initializa Other Variable
averageArea = 0.000  # for calculation of min/max size for contour detected
peopleIn = 0  # number of people going up
peopleOut = 0  # number of people going up
frameCounter = 0
maskT = None
passImage = None
detectedPeople = []
detectedContours = []

# take first frame of the video
_ , pFrame = cap.read()

while (cap.isOpened()):

    # Check Passage Status
    status = cv2.getTrackbarPos(switch, 'config')
    if status == 0:
        allowPassage = True
    else:
        allowPassage = False

    # RE-Initialize
    frameInfo = np.zeros((400, 500, 3), np.uint8)
    averageArea = averageSize()
    ret, frame = cap.read()  # read a frame
    frameForView = frame.copy()

    # Clean Frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    fgmask = fgbg.apply(gray)
    blur = cv2.medianBlur(fgmask, 5)
    thresh = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)[1]  # shadow of MOG@ is grey = 127
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)  # fill any small holes
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel)  # remove noise
    contours = cv2.findContours(opening.copy(), cv2.RETR_EXTERNAL,
                                 cv2.CHAIN_APPROX_SIMPLE)[1]

    mask_opening = cv2.inRange(opening, np.array([0]), np.array([128]))
    noBg = cv2.bitwise_and(frame, frame, mask=mask_opening)

    # Process Contours
    for c in contours:
        # Filter Contour By Size
        if len(humanSizeSample) < 100:
            if cv2.contourArea(c) < minArea or cv2.contourArea(c) > maxArea:
                continue
            else:
                humanSizeSample.append(cv2.contourArea(c))
        else:
            if cv2.contourArea(c) < averageArea/2 or cv2.contourArea(c) > averageArea*3:
                continue
        (x, y, w, h) = cv2.boundingRect(c)
        detectedContours.append(Position(x, y, w, h))

    # Process Detected People
    if len(detectedPeople) != 0:
        for people in detectedPeople:

            # Setup Meanshift/Camshift for Tracking
            track_window = (people.x, people.y, people.w, people.h)
            hsv_roi = pOpening[people.y:people.y + people.h, people.x:people.x + people.w]
            mask = cv2.inRange(hsv_roi, np.array(128), np.array(256))
            roi_hist = cv2.calcHist([hsv_roi], [0], mask, [100], [0, 256])
            cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
            term_criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 1, 1)  # Setup the termination criteria, either 10 iteration or move by atleast 1 pt
            dst = cv2.calcBackProject([opening], [0], roi_hist, [0, 256], 1)
            ret, track_window = cv2.CamShift(dst, track_window, term_criteria)

            # Process POST Tracking
            pts = cv2.boxPoints(ret)
            pts = np.int0(pts)
            img2 = cv2.polylines(frameForView, [pts], True, people.color, 2)
            pos = sum(pts)/len(pts)
            isFound = False
            for dC in detectedContours:
                if dC.x - toleranceRange < pos[0] < dC.x + dC.w + toleranceRange \
                        and dC.y - toleranceRange < pos[1] < dC.y + dC.h + toleranceRange:
                    people.set("x", dC.x)
                    people.set("y", dC.y)
                    people.set("w", dC.w)
                    people.set("h", dC.h)
                    people.set("speed", pos - people.center)
                    people.set("center", pos)
                    people.set("missingCount", 0)
                    detectedContours.remove(dC)
                    isFound = True

                    tR = getExteriorRect(pts)
                    people.set("roi", frame[tR[1]:tR[1]+tR[3], tR[0]:tR[0]+tR[2]])

                    # Process Continuous Tracking
                    prevInStatus = people.isIn
                    currInStatus = checkPosition(boundaryPt1, boundaryPt2, people.center, inCriterion)
                    people.isIn = currInStatus

                    # Check In/Out Status Change
                    if prevInStatus != currInStatus and people.isInChangeFrameCount >= toleranceCountIOStatus:
                        if not allowPassage:
                            passImage = people.roi
                        people.set("isInChangeFrameCount", 0)
                        if currInStatus:
                            peopleIn += 1
                            if not allowPassage:
                                peopleViolationIn += 1
                        else:
                            peopleOut += 1
                            if not allowPassage:
                                peopleViolationOut += 1
                    else:
                        people.set("isInChangeFrameCount", people.isInChangeFrameCount + 1)

            # Process DIS-continuous Tracking
            if not isFound:
                if people.missingCount > toleranceCount:
                    detectedPeople.remove(people)
                else:
                    if checkTouchVSide(people.x + people.speed[0], people.y + people.speed[1], people.w,
                                       people.h, maxWidth, maxHeight, toleranceRange):
                        detectedPeople.remove(people)
                    else:
                        people.set("missingCount", people.missingCount+1)
                        people.set("x", people.x + people.speed[0])
                        people.set("y", people.y + people.speed[1])
                        people.set("center", people.center + people.speed)

    # Check New People
    for dC in detectedContours:
        if checkTouchVSide(dC.x, dC.y, dC.w, dC.h, maxWidth, maxHeight, toleranceRange):
            startHue += hueIncrementValue
            detectedPeople.append(People(dC.x, dC.y, dC.w, dC.h, frame[dC.y:dC.y+dC.h, dC.x:dC.x+dC.w], startHue))

    # RE-set
    detectedContours = []
    pFrame = frame
    pNoBg = noBg
    pOpening = opening
    frameCounter += 1

    # Output
    try:
        # Setup Stats
        textNoOfPeople = "People: " + str(len(detectedPeople))
        textNoIn = "In: " + str(peopleIn)
        textNoOut = "Out: " + str(peopleOut)
        textNoViolationIn = "In: " + str(peopleViolationIn)
        textNoViolationOut = "Out: " + str(peopleViolationOut)

        if allowPassage:
            cv2.line(frameForView, (long(boundaryPt1[0]), long(boundaryPt1[1])),
                     (long(boundaryPt2[0]), long(boundaryPt2[1])), (0, 255, 0), 2)
        else:
            cv2.line(frameForView, (long(boundaryPt1[0]), long(boundaryPt1[1])),
                     (long(boundaryPt2[0]), long(boundaryPt2[1])), (0, 0, 255), 2)

        # Draw Infos
        cv2.putText(frameInfo, textNoOfPeople, (30, 40), cv2.FONT_HERSHEY_SIMPLEX
                    , 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(frameInfo, textNoIn, (30, 80), cv2.FONT_HERSHEY_SIMPLEX
                    , 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(frameInfo, textNoOut, (30, 120), cv2.FONT_HERSHEY_SIMPLEX
                    , 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.line(frameInfo, (0, 160), (640, 160), (255, 255, 255), 1)
        cv2.putText(frameInfo, "VIOLATION", (30, 200), cv2.FONT_HERSHEY_SIMPLEX
                    , 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(frameInfo, textNoViolationIn, (30, 240), cv2.FONT_HERSHEY_SIMPLEX
                    , 1, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(frameInfo, textNoViolationOut, (30, 280), cv2.FONT_HERSHEY_SIMPLEX
                    , 1, (255, 255, 255), 1, cv2.LINE_AA)

        # Display
        cv2.imshow('FrameForView', frameForView)
        # cv2.imshow('Frame', frame)
        if passImage != None:
            cv2.imshow('Violators', passImage)
        cv2.imshow('config', frameInfo)

    except:
        print('EOF')
        break

    # Abort and exit with 'Q' or ESC
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    # else:
    #     cv2.imwrite(chr(k) + ".jpg", frame)

cap.release()
cv2.destroyAllWindows()